# Respuesta a enunciado N°1

In [1]:
#se crea un Dataframe que tenga la variación porcentual anual de personas para las categorías independientes y asalariados 
#importar librerias
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as make_subplots

#se importan las bases de excell
DF_EMP = pd.read_excel('EMPLEO.xlsx')
DF_EXP = pd.read_excel('EXP.xlsx')
DF_IMP = pd.read_excel('IMP.xlsx')
DF_IMACEC = pd.read_excel('IMACEC.xlsx')

#luego filtramos los datos
filterTrimestre = DF_EMP['Trimestre'] == 'Dic - Feb'
df = DF_EMP[filterTrimestre]
6
#se preparan los datos
ano = df['Ano']
independientes = df['Independientes']
asalariados = df['Asalariados']
variacion = {
    'Ano' : ano,
    'independientes' : independientes,
    'asalariados' : asalariados
}

#se crea el DataFrame a partir del dict
variacionDF = pd.DataFrame.from_dict(variacion)

#Calculamos variacion anual 
variacionDF['independientes'] = independientes.pct_change()
variacionDF['asalariados'] = asalariados.pct_change()
print(variacionDF)

      Ano  independientes  asalariados
11   2011             NaN          NaN
23   2012       -0.026396     0.053409
35   2013       -0.007322     0.026924
47   2014        0.084532     0.011679
59   2015       -0.001768     0.015163
71   2016        0.018741     0.014671
83   2017        0.096861    -0.020398
95   2018        0.012505     0.044637
107  2019        0.019451     0.014900
119  2020       -0.013013     0.033235
131  2021       -0.129571    -0.078194


# Respuesta a enunciado N°2

In [2]:
#Usando el DataFrame del IMACEC, se transpone para que las fechas queden en las filas y las categorías como columnas
transpose = DF_IMACEC.pivot_table(columns=['Descrip'])
print(transpose)

Descrip       Comercio      Imacec  Imacec a costo de factores  \
1996-01-01   34.460062   47.104201                   48.262202   
1996-02-01   35.603255   47.845904                   48.948986   
1996-03-01   35.723395   47.850285                   49.061016   
1996-04-01   36.067168   47.850344                   49.026654   
1996-05-01   36.223066   47.902395                   49.062990   
...                ...         ...                         ...   
2021-01-01  125.749300  112.784887                  112.281302   
2021-02-01  125.823288  113.988091                  113.357142   
2021-03-01  124.126904  112.535383                  111.870357   
2021-04-01  122.981002  110.961629                  110.329831   
2021-05-01  139.495050  113.848748                  112.438244   

Descrip     Imacec no minero  Producción de bienes   Servicios  \
1996-01-01         44.348444             64.530047   39.630956   
1996-02-01         44.915919             65.145224   40.208151   
1996-03-0

# Respuesta a enunciado N°3

In [3]:
#Al DataFrame del punto 1 se le agrega la variación porcentual anual del imacec del punto 2 usando un inner join.
# se preparan los datos
measures = {
    'Ano' : [],
    'Imacec' : []
}

dates = transpose.index
for date in dates:
    if(date.day == 1 and date.month == 1):
        measures['Imacec'].append(transpose.loc[date]['Imacec'])
        measures['Ano'].append(date.year)
        pass

# se crea el DataFrame a partir del dict 
imacecDF = pd.DataFrame.from_dict(measures)

# debemos calcular la variacion
imacecDF['Imacec'] = imacecDF['Imacec'].pct_change()

#creamos el join
innerJoin = pd.merge(imacecDF,variacionDF,on='Ano')

#En este caso el primer valor de Imacec no es NaN, a diferencia de indep. y asalariados
innerJoin.loc[innerJoin.Ano == '2011', 'Imacec']  = float('nan')
print(innerJoin)

     Ano    Imacec  independientes  asalariados
0   2011  0.074745             NaN          NaN
1   2012  0.050965       -0.026396     0.053409
2   2013  0.048731       -0.007322     0.026924
3   2014  0.022693        0.084532     0.011679
4   2015  0.026551       -0.001768     0.015163
5   2016  0.017887        0.018741     0.014671
6   2017  0.006489        0.096861    -0.020398
7   2018  0.039132        0.012505     0.044637
8   2019  0.020324        0.019451     0.014900
9   2020  0.013094       -0.013013     0.033235
10  2021 -0.008203       -0.129571    -0.078194


# Respuesta a enunciado 4

In [4]:
# Utilizando plotly se crea un subplot que contenga:
#Gráfico con la variación anual de independientes,asalariados e IMACEC. 
#se crea un gráfico de scatter que tenga el total de exportaciones en el eje horizontal y el total de importaciones 
#(CIF) en el vertical.
#Dentro de las exportaciones se seleccionará: total exportaciones, minería, agropecuario-silvícola y pesquero, 
#industriales. Utilizando la primera fecha (enero-2009) como base 100, se crea un mapa de calor que en el eje 
#horizontal tenga las fechas y en el vertical las 4 categorías.

In [11]:
#importamos plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

'''
graph1
'''
fig = make_subplots(rows=2, cols=2)

#se preparan los datos
imacecData = go.Scatter(
    x=innerJoin['Ano'],
    y=innerJoin["Imacec"],
    name="Imacec",
    line_color='darkviolet')

asalariadosData = go.Scatter(
    x=innerJoin['Ano'],
    y=innerJoin["asalariados"],
    name="Asalariados",
    line_color= 'maroon')

indepData = go.Scatter(
    x=innerJoin['Ano'],
    y=innerJoin["independientes"],
    name="Independientes",
    line_color= 'cyan')

data = [imacecData, asalariadosData, indepData]

#se grafica
layout = go.Layout(
    yaxis=dict(
        domain=[0, 1]
    ),
    legend=dict(
        traceorder="reversed"
    ),
    yaxis2=dict(
        domain=[0.33, 0.66]
    ),
    yaxis3=dict(
        domain=[0.66, 1]
    ),
    width=2000, height=800,
    title = "Variacion Anual de independientes, asalariados e Imacec",
    xaxis_title="Fecha",
    yaxis_title="Variacion Anual")

fig = go.Figure(data=data, layout=layout)
fig.show()

'''
graph2
'''
#debemos filtrar los datos
impData = DF_IMP.iloc[0].iloc[2:]
expData = DF_EXP.iloc[0].iloc[2:]

#se prepran los datos
dataDict = {
    'imp' : impData,
    'exp' : expData
}
dataDF = pd.DataFrame.from_dict(dataDict)

#se grafica
fig2 = px.scatter(
    dataDF,
    x="exp",
    y="imp",
    title = "Scatter Exportaciones vs Importaciones ",
    width=800, height=800,
    color_discrete_sequence= ['#ffa500']
)
fig2.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
fig2.show()

'''
graph3
'''
#filtramos los datos
dataRaw = DF_EXP.loc[[0,1,11,26]]

#se preparan los datos
heatMapDict = {
    'categorias' : dataRaw['Descripción series'],
    'fechas' : dataRaw.columns[2:]
}
print(px.colors.qualitative.Plotly)
#graficamos
fig3 = px.imshow(
    dataRaw,
    x=dataRaw.columns,
    y=dataRaw['Descripción series'],
    text_auto=True,
    aspect="auto",
    labels=dict(x="Fechas", y="Categorias", color="Exportaciones"),
    width=1200, height=800,
    color_continuous_scale ='solar'
    )
fig3.show()

['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
